In [2]:
import pandas as pd
import time
start = time.time()

df = pd.read_csv('D:\Programowanie\Projekty\chess_cheater_project\datasets\chess_games.csv',nrows=1_000)
df['AN']

0      1. d4 d5 2. c4 c6 3. e3 a6 4. Nf3 e5 5. cxd5 e...
1      1. e4 e5 2. b3 Nf6 3. Bb2 Nc6 4. Nf3 d6 5. d3 ...
2      1. e4 d5 2. exd5 Qxd5 3. Nf3 Bg4 4. Be2 Nf6 5....
3      1. e3 Nf6 2. Bc4 d6 3. e4 e6 4. Nf3 Nxe4 5. Nd...
4      1. e4 c5 2. Nf3 d6 3. d4 cxd4 4. Nxd4 Nf6 5. N...
                             ...                        
995    1. d4 d5 2. Nf3 Nc6 3. e3 Nf6 4. Bd3 e6 5. Nbd...
996    1. d4 e6 2. e4 c5 3. d5 d6 4. Nc3 f5 5. dxe6 B...
997    1. e4 e6 2. d4 c5 3. c3 a6 4. Nf3 h6 5. Be2 Be...
998    1. e3 { [%eval 0.1] } 1... c5 { [%eval 0.18] }...
999    1. e4 c6 2. Nc3 Nf6 3. g3 d6 4. Bg2 g6 5. Nge2...
Name: AN, Length: 1000, dtype: object

In [3]:
import re

def moves_to_list(moves):
    moves_list = []
    for element in moves.split():
        if re.match('[a-zA-Z]+[1-9]|O-O|O-O-O', element) is not None:
            element = element.replace('?', '')
            element = element.replace('!', '')
            element = element.replace('+', '')

            moves_list.append(element)
    
    return moves_list

df['move_list'] = df['AN'].apply(moves_to_list)
df['move_list']

0      [d4, d5, c4, c6, e3, a6, Nf3, e5, cxd5, e4, Ne...
1      [e4, e5, b3, Nf6, Bb2, Nc6, Nf3, d6, d3, g6, N...
2      [e4, d5, exd5, Qxd5, Nf3, Bg4, Be2, Nf6, Nc3, ...
3      [e3, Nf6, Bc4, d6, e4, e6, Nf3, Nxe4, Nd4, Nxf...
4      [e4, c5, Nf3, d6, d4, cxd4, Nxd4, Nf6, Nc3, a6...
                             ...                        
995    [d4, d5, Nf3, Nc6, e3, Nf6, Bd3, e6, Nbd2, Be7...
996    [d4, e6, e4, c5, d5, d6, Nc3, f5, dxe6, Bxe6, ...
997    [e4, e6, d4, c5, c3, a6, Nf3, h6, Be2, Be7, O-...
998    [e3, c5, Ne2, Nc6, Ng3, d6, Be2, e5, O-O, f5, ...
999    [e4, c6, Nc3, Nf6, g3, d6, Bg2, g6, Nge2, Bg7,...
Name: move_list, Length: 1000, dtype: object

In [4]:
import chess
import numpy as np


def save_board_state_after_move(moves):
    board = chess.Board()
    FEN_list = [board.fen()]
    try:
        for move in moves:
            board.push_san(move)
            FEN_list.append(board.fen())
        return FEN_list
    except Exception as e:
        print(e)
        return np.nan

df['FENs'] = df['move_list'].apply(save_board_state_after_move)

In [5]:
exploded_FENs = df['FENs'].explode()

In [6]:
data = {'index' : exploded_FENs.index, 'FEN' : exploded_FENs.values}
df_games = pd.DataFrame(data=data)

In [7]:
df_games['move'] = df_games.groupby('index')['index'].rank(method='first').astype('int')

In [8]:
list_for_threads = []
for i in range(1, 9):
    list_for_threads.append(df_games[(df_games['index'] < i * 125) & (df_games['index'] >= (i-1) * 125)])

In [9]:
from stockfish import Stockfish
# from random import randint

def generate_stockfish_moves(FEN):
    FENs_list = []
    stockfish = Stockfish(path="D:\Programowanie\StockFish\stockfish_15.1_win_x64_avx2\stockfish-windows-2022-x86-64-avx2.exe")
    board = chess.Board(FEN)
    stockfish.set_fen_position(FEN)
    for i in range(1):
        if board.is_checkmate():
            break
        stockfish.set_fen_position(board.fen())
        top_moves = stockfish.get_top_moves(3)
        # selected_move = randint(0, len(top_moves)-1)
        # move = top_moves[selected_move]['Move']
        try:
            for move in top_moves:
                board = chess.Board(FEN)
                board.push_uci(move['Move'])
                FENs_list.append(board.fen())
        except Exception as e:
            print(e)
            break
    return FENs_list

In [10]:
from threading import Thread

class CustomThread(Thread):
    def __init__(self, target, df):
        Thread.__init__(self)
        self.target = target
        self.df = df
        self.value = None
 
    def run(self):
        self.value = self.target(self.df)

In [11]:
def run_generate_stockfish_moves(df):
    return_dict_list = []
    for idx, iter_row in enumerate(df.iterrows()):
        _, row = iter_row
        return_dict = {}
        return_dict['FEN'] = generate_stockfish_moves(row['FEN'])
        return_dict['move'] = row['move']
        return_dict['index'] = row['index']
        return_dict_list.append(return_dict)
        print(idx/len(df) * 100, end='\r')

    return return_dict_list

In [12]:
stockfish_fens_list = []
threads = list()
for idx, list_for_thread in enumerate(list_for_threads):
    thread = CustomThread(target=run_generate_stockfish_moves, df=list_for_thread)
    threads.append(thread)
    thread.start()
    # print(idx)

for thread in threads:
    thread.join()
    stockfish_fens_list.append(thread.value)

In [13]:
stockfish_fens_list_union = []
for fens_list in stockfish_fens_list:
    stockfish_fens_list_union += fens_list
stockfish_fens_list_union[0]

{'FEN': ['rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1',
  'rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R b KQkq - 1 1',
  'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq - 0 1'],
 'move': 1,
 'index': 0}

In [14]:
df_stockfish_moves = pd.DataFrame(data=stockfish_fens_list_union)

In [15]:
df_stockfish_moves['move'] = df_stockfish_moves['move'] + 1

In [16]:
df_games

,index,FEN,move
0,0,rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w ...,1
1,0,rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...,2
2,0,rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBN...,3
3,0,rnbqkbnr/ppp1pppp/8/3p4/2PP4/8/PP2PPPP/RNBQKBN...,4
4,0,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKB...,5
...,...,...,...
69012,999,r4rk1/pp2ppbp/1np1bnp1/8/5P2/2N3P1/PPPRN1BP/R1...,27
69013,999,r4rk1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/R...,28
69014,999,r2r2k1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/...,29
69015,999,r2r2k1/pp2ppbp/1np1bnp1/8/5P2/BPN3P1/P1PRN1BP/...,30


In [28]:
df_data = pd.merge(df_games, df_stockfish_moves, on=['index', 'move'], how='left', suffixes=['', '_stockfish'])
# df_data = df_data.dropna()
df_data

,index,FEN,move,FEN_stockfish
1,0,rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...,2,[rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR...
2,0,rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBN...,3,[rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKB...
3,0,rnbqkbnr/ppp1pppp/8/3p4/2PP4/8/PP2PPPP/RNBQKBN...,4,[rnbqkbnr/ppp1pppp/8/3p4/2PP4/8/PP2PPPP/RNBQKB...
4,0,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKB...,5,[rnbqkbnr/ppp2ppp/4p3/3p4/2PP4/8/PP2PPPP/RNBQK...
5,0,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/4P3/PP3PPP/RNBQK...,6,[rnbqkbnr/pp2pppp/2p5/3P4/3P4/8/PP2PPPP/RNBQKB...
...,...,...,...,...
69012,999,r4rk1/pp2ppbp/1np1bnp1/8/5P2/2N3P1/PPPRN1BP/R1...,27,[r1b2rk1/pp2ppbp/2p2np1/8/2n2P2/2N3P1/PPPRN1BP...
69013,999,r4rk1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/R...,28,[r4rk1/pp2ppbp/1np1bnp1/8/3N1P2/2N3P1/PPPR2BP/...
69014,999,r2r2k1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/...,29,[r4rk1/pp2ppbp/1np1b1p1/3n4/5P2/1PN3P1/P1PRN1B...
69015,999,r2r2k1/pp2ppbp/1np1bnp1/8/5P2/BPN3P1/P1PRN1BP/...,30,[r2R2k1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1P1N1BP...


In [29]:
df_data[['FEN_stockfish_1', 'FEN_stockfish_2', 'FEN_stockfish_3']] = pd.DataFrame(data=df_data['FEN_stockfish'].tolist(), index=df_data.index)

In [30]:
df_data = df_data.drop('FEN_stockfish', axis=1)

In [31]:
df_data['FEN'].iloc[0]

'rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1'

In [33]:
df_data['FEN_stockfish_1'].iloc[0]

'rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR b KQkq - 0 1'

In [34]:
df_data[(df_data['FEN'] != df_data['FEN_stockfish_1']) &
        (df_data['FEN'] != df_data['FEN_stockfish_2']) &
        (df_data['FEN'] != df_data['FEN_stockfish_3'])
        ]

,index,FEN,move,FEN_stockfish_1,FEN_stockfish_2,FEN_stockfish_3
5,0,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/4P3/PP3PPP/RNBQK...,6,rnbqkbnr/pp2pppp/2p5/3P4/3P4/8/PP2PPPP/RNBQKBN...,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/2N5/PP2PPPP/R1BQ...,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/5N2/PP2PPPP/RNBQ...
6,0,rnbqkbnr/1p2pppp/p1p5/3p4/2PP4/4P3/PP3PPP/RNBQ...,7,rn1qkbnr/pp2pppp/2p5/3p1b2/2PP4/4P3/PP3PPP/RNB...,rnbqkb1r/pp2pppp/2p2n2/3p4/2PP4/4P3/PP3PPP/RNB...,rnbqkbnr/pp3ppp/2p1p3/3p4/2PP4/4P3/PP3PPP/RNBQ...
8,0,rnbqkbnr/1p3ppp/p1p5/3pp3/2PP4/4PN2/PP3PPP/RNB...,9,rn1qkbnr/1p2pppp/p1p5/3p1b2/2PP4/4PN2/PP3PPP/R...,rnbqkb1r/1p2pppp/p1p2n2/3p4/2PP4/4PN2/PP3PPP/R...,rnbqkbnr/1p2pp1p/p1p3p1/3p4/2PP4/4PN2/PP3PPP/R...
9,0,rnbqkbnr/1p3ppp/p1p5/3Pp3/3P4/4PN2/PP3PPP/RNBQ...,10,rnbqkbnr/1p3ppp/p1p5/3pP3/2P5/4PN2/PP3PPP/RNBQ...,rnbqkbnr/1p3ppp/p1p5/3pN3/2PP4/4P3/PP3PPP/RNBQ...,rnbqkbnr/1p3ppp/p1p5/3pp3/2PPP3/5N2/PP3PPP/RNB...
20,0,r2qk2r/1p1nbppp/p4n2/3p4/3Pp3/1QN1P3/PP3PPP/R1...,21,r3kb1r/1p1n1ppp/pq3n2/3p4/3Pp3/1QN1P3/PP3PPP/R...,r2qkb1r/1p3ppp/pn3n2/3p4/3Pp3/1QN1P3/PP3PPP/R1...,r2qkb1r/3n1ppp/p4n2/1p1p4/3Pp3/1QN1P3/PP3PPP/R...
...,...,...,...,...,...,...
69005,999,r1bq1rk1/pp2ppbp/2p2np1/4n3/5P2/2N3P1/PPP1N1BP...,20,r1bq1rk1/pp2ppbp/2p2np1/4n3/8/2N3PP/PPP1NPB1/R...,r1bq1rk1/pp2ppbp/2p2np1/4n3/5B2/2N3P1/PPP1NPBP...,r1bq1rk1/pp2ppbp/2p2np1/4n3/5N2/2N3P1/PPP2PBP/...
69007,999,r1bq1rk1/pp1nppbp/2p2np1/8/5P2/2N3P1/PPPQN1BP/...,22,r1bq1rk1/pp1nppbp/2p2np1/8/P4P2/2N3P1/1PP1N1BP...,r1bq1rk1/pp1nppbp/2p2np1/8/5P2/2N3PP/PPP1N1B1/...,r1bq1rk1/pp1nppbp/2p2np1/8/5P2/2N3P1/PPP1N1BP/...
69012,999,r4rk1/pp2ppbp/1np1bnp1/8/5P2/2N3P1/PPPRN1BP/R1...,27,r1b2rk1/pp2ppbp/2p2np1/8/2n2P2/2N3P1/PPPRN1BP/...,r1b2rk1/pp2ppbp/1np3p1/8/5Pn1/2N3P1/PPPRN1BP/R...,r1b2rk1/1p2ppbp/1np2np1/p7/5P2/2N3P1/PPPRN1BP/...
69013,999,r4rk1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/R...,28,r4rk1/pp2ppbp/1np1bnp1/8/3N1P2/2N3P1/PPPR2BP/R...,r4rk1/pp2ppbp/1np1bnp1/8/5P2/2N3P1/PPP1N1BP/R1...,r4rk1/pp2ppbp/1np1bnp1/8/P4P2/2N3P1/1PPRN1BP/R...


In [35]:
df_data

,index,FEN,move,FEN_stockfish_1,FEN_stockfish_2,FEN_stockfish_3
1,0,rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...,2,rnbqkbnr/pppppppp/8/8/3P4/8/PPP1PPPP/RNBQKBNR ...,rnbqkbnr/pppppppp/8/8/8/5N2/PPPPPPPP/RNBQKB1R ...,rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR ...
2,0,rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBN...,3,rnbqkbnr/ppp1pppp/8/3p4/3P4/8/PPP1PPPP/RNBQKBN...,rnbqkb1r/pppppppp/5n2/8/3P4/8/PPP1PPPP/RNBQKBN...,rnbqkbnr/pppp1ppp/4p3/8/3P4/8/PPP1PPPP/RNBQKBN...
3,0,rnbqkbnr/ppp1pppp/8/3p4/2PP4/8/PP2PPPP/RNBQKBN...,4,rnbqkbnr/ppp1pppp/8/3p4/2PP4/8/PP2PPPP/RNBQKBN...,rnbqkbnr/ppp1pppp/8/3p4/3P4/5N2/PPP1PPPP/RNBQK...,rnbqkbnr/ppp1pppp/8/3p4/3P1B2/8/PPP1PPPP/RN1QK...
4,0,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKB...,5,rnbqkbnr/ppp2ppp/4p3/3p4/2PP4/8/PP2PPPP/RNBQKB...,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/8/PP2PPPP/RNBQKB...,rnbqkbnr/ppp1pppp/8/8/2pP4/8/PP2PPPP/RNBQKBNR ...
5,0,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/4P3/PP3PPP/RNBQK...,6,rnbqkbnr/pp2pppp/2p5/3P4/3P4/8/PP2PPPP/RNBQKBN...,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/2N5/PP2PPPP/R1BQ...,rnbqkbnr/pp2pppp/2p5/3p4/2PP4/5N2/PP2PPPP/RNBQ...
...,...,...,...,...,...,...
69012,999,r4rk1/pp2ppbp/1np1bnp1/8/5P2/2N3P1/PPPRN1BP/R1...,27,r1b2rk1/pp2ppbp/2p2np1/8/2n2P2/2N3P1/PPPRN1BP/...,r1b2rk1/pp2ppbp/1np3p1/8/5Pn1/2N3P1/PPPRN1BP/R...,r1b2rk1/1p2ppbp/1np2np1/p7/5P2/2N3P1/PPPRN1BP/...
69013,999,r4rk1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/R...,28,r4rk1/pp2ppbp/1np1bnp1/8/3N1P2/2N3P1/PPPR2BP/R...,r4rk1/pp2ppbp/1np1bnp1/8/5P2/2N3P1/PPP1N1BP/R1...,r4rk1/pp2ppbp/1np1bnp1/8/P4P2/2N3P1/1PPRN1BP/R...
69014,999,r2r2k1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/...,29,r4rk1/pp2ppbp/1np1b1p1/3n4/5P2/1PN3P1/P1PRN1BP...,r4rk1/pp2ppbp/2p1bnp1/3n4/5P2/1PN3P1/P1PRN1BP/...,r2r2k1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/...
69015,999,r2r2k1/pp2ppbp/1np1bnp1/8/5P2/BPN3P1/P1PRN1BP/...,30,r2R2k1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1P1N1BP/...,r2r2k1/pp2ppbp/1np1bnp1/8/5P2/1PN3P1/P1PRN1BP/...,r2r2k1/pp2ppbp/1np1bnp1/8/P4P2/1PN3P1/2PRN1BP/...


In [36]:
df_data.to_csv('top3_stockfish_moves_for_every_fen.csv')